__<font color='red'>
Note: This feature is currently under development so it will not be fully functional yet. 
Also, the APIs here are experimental and subject to change.
</font>__

## Setup and loading traces
To run this demo notebook on your laptop
1. Clone the repo `git clone https://github.com/facebookresearch/HolisticTraceAnalysis.git`
1. [Optional and recommended] Setup a conda environment. See README for details.
1. Set the `trace_dir` parameter in the next cell to the location of `tests/data/critical_path/simple_add` folder in your local `HolisticTraceAnalysis` installation.

In [1]:
from hta.trace_analysis import TraceAnalysis

trace_dir = "/Users/bcoutinho/Work/hta/HolisticTraceAnalysis2/tests/data/critical_path/simple_add/"

analyzer = TraceAnalysis(trace_dir=trace_dir)

ProfilerStep not found in the trace. The analysis result may not be accurate.


# Critical Path Analysis

Critical path analysis is a commonly applied technique in HPC and AI/ML optimization. It can be leveraged in two ways:
1. **Efficiency opportunities** Operations/kernels on critical path should be the target of performance analysis and optimizations. They can provide the “best bang for buck” for performance improvements.This is not limited to just CPU/GPU kernels. Delays in launching or executing CUDA kernels can constitute a significant portion of the critical path as well. This could be optimized by operator fusion and CUDA graphs for example.
2. **Simulating improvements / gains:** After identifying critical path we can estimate improvements to the execution time without actually running anything. This helps us estimate the possible gains from various performance optimization.

## Approach
In a nutshell, computing the critical path involves 1) constructing a weighted DAG connecting all the operations, 2) finding the longest path in this DAG. The challenging part is constructing the DAG here. 

**Nodes**: The Nodes in the critical path graph represent points in time. Each operator/kernel thus has two nodes viz. a begin and an end node. In case of nested operators we also link the nodes in the order they appear in the callstack. An example of stacked operators is shown below-
```
          |----------------------- Op A ----------------------|
                    |--- Op B ---|        |-- Op C--|
Critical Path Graph
         (OpA.b)--->(ObB.b)----->(OpB.e)->(OpC.b)-->(OpC.e)->(OpA.e)
```

**Edges** in this DAG can be one of two types
1. Timing edges (weight = time): include durations for the operators/kernels as well as delays to launch operators between CPU and GPU.
1. Dependency edges (weight = 0): do not have a time component but show a dependency between operations themselves. This includes data dependencies and synchronization between CPU and GPU.

## Lightweight Critical Path
Our initial implementation of Critical Path analysis does not consider dependency between PyTorch operators, this requires combining the trace with Chakra Execution Traces and processing Tensor information.

The key idea here is to simplify the dependency analysis between PyTorch operators
- We assume that all CPU operators are dependent serially on the last operator that ran on a CPU thread. This frees us from attempting to correlate with Execution Traces.
- The operator dependency part can be added back later.




## Using this notebook

We can demonstrate critical path analysis on most PyTorch traces. However, currently traces are missing
information regarding CUDA synchronization. 

The above is fixed in this [PR](https://github.com/pytorch/pytorch/pull/105187) but not enabled by default. Please follow the documentation in this [PR](https://github.com/pytorch/pytorch/pull/105187) to enable CUDA synchronization events to get best results from this analysis.

In [2]:
analyzer.critical_path_analysis?

Signature:
analyzer.critical_path_analysis(
    rank: int,
    annotation: str,
    instance_id: Union[int, NoneType, Tuple[int, int]],
) -> Tuple[hta.analyzers.critical_path_analysis.CPGraph, bool]
Docstring:
Perform critical path analysis for trace events within a rank.
We further reduce the region of interest by selecting
a trace annotation and instance id. This will
limit the analysis to events within the time range of that annoation.
This will include GPU kernels launched by the cpu operators in that
time duration.
For example, you can use this to limit the analysis to one iteration
by passing annotation='ProfilerStep'. See notes for how to pick the iteration.

Args:
    t (Trace): Input trace data structure.
    rank (int): rank to analyze for the critical path.
    annotation (str): a trace annotation to limit the analysis to,
        for example "ProfilerStep" would match all annotations that
        match this string (ProfilerStep100, ProfilerStep101 etc)
    instance_id: can 

In [3]:
annotation = "[param|pytorch.model.alex_net|0|0|0|measure|forward]"
instance_id = 1  # note this is zero based

# You can also pass a tuple specifying range of annotation instances
# annotation = "ProfilerStep"  # will match multiple ProfilerStepXXX annotations
# instance_id = (2,3)          # include 3rd to 4th iteration.

In [4]:
cp_graph, success = analyzer.critical_path_analysis(
    rank = 0, annotation=annotation, instance_id=instance_id)
success                                             

True

## Overlay and visualize the Critical Path
The `overlay_critical_path_analysis()` function exposes the critical path on the original trace file.
There are two modes for the output:
1. When `only_show_critical_events=True` (default value) the output trace only contains CPU operators and GPU events on the critical path. One can compare it with the original trace to contrast the critical path identified by the algorithm.
1. When `only_show_critical_events=False` in the output trace file search for "critical" to highlight events on the critical path.

Edges in the critical path graph will be shown using arrows or flow events. Critical edges are marked using the "critical" flag in the args property.

The category names of flow events begin with "critical_path_". They have the following meaning:
* `critical_path_dependency`: an inter operator dependency.
* `critical_path_operator`: an edge between start and end of a CPU operator or GPU kernel.
* `critical_path_kernel_launch_delay`: delay to launch a GPU kernel that is likely to be on the critical path.
* `critical_path_kernel_kernel_delay`: delay between running successive GPU kernels.
* `critical_path_sync_dependency`: these edges denote synchronization or control dependencies between events such as GPU->CPU, GPU->GPU synchronization.

In [5]:
analyzer.overlay_critical_path_analysis?

Signature:
analyzer.overlay_critical_path_analysis(
    rank: int,
    critical_path_graph: hta.analyzers.critical_path_analysis.CPGraph,
    output_dir: str,
    only_show_critical_events: bool = True,
    show_all_edges: bool = False,
) -> str
Docstring:
Overlay the identified critical path on top of the trace file
for visualization.

Args:
    rank (int): rank to generate the time series for.
    critical_path_graph: Critical Path Graph object generated previously
    output_dir (str): Output directory to store overlaid trace.
    only_show_critical_events (bool): When set the output trace will only
        have operators and GPU kernels on the critical path. It will
        still retain the user annotations.
    show_all_edges (bool): When set this will add edge events for
        all types of edges in the critical path graph. This is useful
        for debugging the algorithm.

Returns: the overlaid trace file path. The generated trace file will
have a prefix of "overlaid_critical

In [6]:
# Make sure '~/HolisticTraceAnalysis/tests/data/critical_path/simple_add/overlaid' exists or add a different directory
# to write your trace too.
analyzer.overlay_critical_path_analysis(
    0, cp_graph, output_dir='~/HolisticTraceAnalysis/tests/data/critical_path/simple_add/overlaid')

'/Users/bcoutinho/Work/hta/critical_path/simple_add/overlaid/overlaid_critical_path_benchmark_result_493459_1694039959_trace.json.gz'

## More details on the CPGraph object

In [ ]:
from hta.analyzers.critical_path_analysis import CPGraph
CPGraph?